In [1]:
import pandas as pd
import numpy as np
import re

In [3]:
def evaluate_rules(data, tuned_rules, output):
    predictions_all = []
    for i, rule in tuned_rules.iterrows():
        pred_rule = []
        tptot = 0
        tntot = 0
        fntot = 0
        fptot = 0
        tuned_antecedent = rule['Rule'].strip()
        print(f"rule: {tuned_antecedent}")
        for _, row in data.iterrows():
            y_row = row[output]
            tp = 0
            tn = 0
            fn = 0
            fp = 0
            # check if the point row satifies rule 
            if evaluate_rule_conditions(row, tuned_antecedent):
                # rule is satisfied
                satisfied = True
                pred_rule.append(0)
            else:
                satisfied = False
                pred_rule.append(1)
    
            
            if satisfied and y_row == 0:
                tp=1
            elif satisfied and y_row == 1:
                fp=1
            elif not satisfied and y_row == 1:
                tn=1
            
            elif not satisfied and y_row == 0:
                fn=1

            predictions_all.append(pred_rule)
            tptot+=tp
            tntot+=tn
            fptot+=fp
            fntot+=fn

        if (tptot+fptot) == 0:
            print("no points satisfy the rule\n")
            continue
        else:
            
            print(f"tp = {tptot}, tn = {tntot}, fp = {fptot}, fn = {fntot}")
            prec = tptot / (tptot+fptot)
            cov = tptot/(tptot+fntot)
            acc = (tptot+tntot)/(tptot+tntot+fptot+fntot)
            f1score = (2*tptot)/(2*tptot+fptot+fntot)
            error = fptot/(fptot+tntot)
            print(f"precision: {prec}, covering: {cov}, error: {error}, accuracy: {acc}, f1: {f1score}\n")              

    return predictions_all


def evaluate_rule_conditions(row, condition_part):

    # Checks if any of the conditions in the rule are satisfied
    if all(check_condition(row, part) for part in condition_part.split(" and ")):
        return True  # Return True if all conditions in the rule are satisfied
    
    return False  # Return False if any of the conditions in the rule is not satisfied


def check_condition(row, condition_part):
    # Check if a single condition part is satisfied
    
    #parts = [part.strip('()') for part in condition_part.split()]
    parts = re.split(r'\s*(==|<=|>=|<|>|!=)\s*', condition_part.strip('()'))
    #print("parts: ", parts)
    if len(parts) == 3:
        column, op, value = parts
        return eval(f"{row[column]} {op} {value}")
    # handle the case of a 2-thresholds conditions of the kind: a < Column <= b
    elif len(parts) == 5:
        val1,op1,column,op2,val2 = parts
        # Use the original condition from the rule
        return eval(f"{val1} {op1} {row[column]} {op2} {val2}")        
    else:
        # Handle cases where the condition is not in the expected format
        pass


# Skope Rules performance

In [4]:
#mix both rules, model for collision and model for no collision.
skrules_col = pd.read_csv('simulationVictor/skope/skope_rules_collisions.csv', header=None, names=["Rule","Covering","Error"])
skrules_nocol = pd.read_csv('simulationVictor/skope/skope_rules_noncollisions.csv', names=["Rule","Covering","Error"])

skrules = pd.concat([skrules_col, skrules_nocol], ignore_index=True)
skrules

,Rule,Covering,Error
0,RULE 1: IF SafetyMargin <= 0.04231328144669533...,COVERING: 0.48856416772554,ERROR: 0.04353233830845771
1,RULE 2: IF SafetyMargin <= 0.03577385842800140...,COVERING: 0.47365079365079366,ERROR: 0.03116883116883118
2,RULE 3: IF SafetyMargin <= 0.04231328144669533...,COVERING: 0.16805644644002565,ERROR: 0.16025641025641024
3,RULE 4: IF SafetyMargin <= 0.04231328144669533...,COVERING: 0.11912427559562137,ERROR: 0.13145539906103287
4,RULE 1: IF SafetyMargin <= 0.04231328144669533...,COVERING: 0.48856416772554,ERROR: 0.04353233830845771
5,RULE 2: IF SafetyMargin <= 0.03577385842800140...,COVERING: 0.47365079365079366,ERROR: 0.03116883116883118
6,RULE 3: IF SafetyMargin <= 0.04231328144669533...,COVERING: 0.16805644644002565,ERROR: 0.16025641025641024
7,RULE 4: IF SafetyMargin <= 0.04231328144669533...,COVERING: 0.11912427559562137,ERROR: 0.13145539906103287
8,RULE 1: IF SafetyMargin > 0.041092799976468086...,COVERING: 0.5300245098039216,ERROR: 0.039955604883462836
9,RULE 2: IF SafetyMargin > 0.041092799976468086...,COVERING: 0.4701627486437613,ERROR: 0.02621722846441943


In [10]:
# adjust columns values
for i in range(len(skrules)+1):   
    skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace("RULE {}: ".format(i),""))
skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace("AND","and"))
skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace("{",""))
skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace("}",""))
#skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace("output in ","output = "))
skrules["Covering"] = skrules["Covering"].apply(lambda x: x.replace("COVERING: ",""))
skrules["Error"] = skrules["Error"].apply(lambda x: x.replace("ERROR: ",""))
skrules['Output'] = skrules['Rule'].str.extract(r'output = (\d+)', expand=False).astype(int)
skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace("IF ",""))
skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace(x[x.find("THEN"):],""))


data = pd.read_csv("simulationVictor/test.csv")
skrules_noncoll = skrules[skrules["Output"]==0]
preds_skope = evaluate_rules(data, skrules_noncoll, "output")

rule: SafetyMargin > 0.040334392338991165 and Tau <= 0.4974003732204437
tp = 289, tn = 24, fp = 2, fn = 233
precision: 0.993127147766323, covering: 0.553639846743295, error: 0.07692307692307693, accuracy: 0.5711678832116789, f1: 0.7109471094710947

rule: SafetyMargin > 0.0397886261343956 and Eta > 0.30642834305763245 and Tau <= 0.6090118587017059
tp = 267, tn = 25, fp = 1, fn = 255
precision: 0.996268656716418, covering: 0.5114942528735632, error: 0.038461538461538464, accuracy: 0.5328467153284672, f1: 0.6759493670886076

rule: SafetyMargin > 0.06201409734785557 and Tau > 0.6082371175289154
tp = 78, tn = 18, fp = 8, fn = 444
precision: 0.9069767441860465, covering: 0.14942528735632185, error: 0.3076923076923077, accuracy: 0.17518248175182483, f1: 0.2565789473684211

rule: SafetyMargin > 0.0397886261343956 and Eta <= 0.3262890875339508 and Tau <= 0.6082371175289154
tp = 100, tn = 23, fp = 3, fn = 422
precision: 0.970873786407767, covering: 0.19157088122605365, error: 0.11538461538461539